In [ ]:
!cp -r '/content/drive/MyDrive/TFG/HTG/data' './data'
!cp -r '/content/drive/MyDrive/TFG/HTG/models' './models'
!cp -r '/content/drive/MyDrive/TFG/HTG/options' './options'
!cp -r '/content/drive/MyDrive/TFG/HTG/util' './util'
!cp '/content/drive/MyDrive/TFG/HTG/input/PerfectSQL_2.sql' .
!cp '/content/drive/MyDrive/TFG/HTG/latest_net_G.pth' .

In [ ]:
from PIL import Image
import torch
import numpy as np
from options.test_options import TestOptions
from models.BigGAN_networks import Generator
from models.OCR_network import strLabelConverter
from data.alphabets import alphabetEnglish, alphabetFrench
from util.util import prepare_z_y
import os
import threading
import time

n_threads = 0
seeds = [0 for i in range(5)]

class myThread (threading.Thread):
    def __init__(self, threadID, word, l, skiplines, w, i):
        threading.Thread.__init__(self)
        self.word = word
        self.l = l
        self.skiplines = skiplines
        self.w = w
        self.i = i
        self.threadID = threadID
    def run(self):
        generate_img_thread(self.threadID, self.word, self.l, self.skiplines, self.w, self.i)


def load_model():
    opt = TestOptions().parse()  # get test options
    opt.n_classes = 80
    gen = Generator(**vars(opt))

    #state_dict = torch.load('checkpoints/_IAMcharH32W16rmPunct_GANres16_bs8/latest_net_G.pth')
    state_dict = torch.load('latest_net_G.pth')
    if hasattr(state_dict, '_metadata'):
        del state_dict._metadata
    gen.load_state_dict(state_dict)
    gen.eval()
    return gen

model = load_model()

char_to_int = {v:k for k, v in enumerate(alphabetEnglish)}


def get_word(word):
    encoded = [char_to_int[char] for char in word]
    words = torch.zeros((1, len(encoded), 80), dtype=torch.float32)
    for i, code in enumerate(encoded):
        words[0, i, code] = 1
    return words

def generate_image(word, threadID):
    seed = seeds[threadID]
    #seed = np.random.randint(0, 10e4)
    words = get_word(word)
    z, _ = prepare_z_y(1, 128, 80, device='cpu', seed=seed) #TODO Change device='cpu' to device='cuda' for Google Colab
    res = model.forward(z=z, y=words)
    res = res.detach().numpy()[0, 0] * 255
    im = Image.fromarray(res).convert('RGB')
    return im

def generate_img_thread(threadID, word, l, skiplines, w, i):
    global n_threads
    img = generate_image(word, threadID)
    img.save("output/line" + str(l+skiplines) + "/word" + str(w) + "_img" + str(i) + ".png", "PNG")
    n_threads -= 1


if __name__ == '__main__':
    global seeds
    import sys
    from tqdm import tqdm
    skiplines = 0
    #if len(sys.argv)==3:
    #    skiplines = int(sys.argv[2])                              # Number of lines to be skipped
    nlineas = len(open('./PerfectSQL_2.sql', 'r').readlines()[skiplines:]) # Number of lines (removing the number of lines to be skipped)

    f = open('./PerfectSQL_2.sql', 'r')  # f = input file

    for s in range(skiplines):
        f.readline(1)	            # Skip the first N lines of the file if needed (1 means only 1 char of the line is read, to speed it up)
    line = f.readline()             # Read the whole line
    for l in tqdm(range(nlineas)):  # For each line
        if line != '':
            words = line.split()
            w = 0
            for s in range(5):
              seeds[s] = np.random.randint(0, 10e4)
            for word in words:      # For each word
                while n_threads > 0:
                    time.sleep(0.1)
                if not os.path.exists("output/line"+str(l+skiplines)):
                    os.makedirs("output/line"+str(l+skiplines))
                for i in range(0,5):    # Process word 5 times (multiple handwritting styles)
                    n_threads += 1
                    myThread(i, word, l, skiplines, w, i).start()
                    #img = generate_image(word)
                    #img.save("output/line" + str(l+skiplines) + "/word" + str(w) + "_img" + str(i) + ".png", "PNG")
                w += 1
#            l += 1
        line = f.readline()


#    text = 
#    for i in range(0, 10):    
#        img = generate_image(text)
#    img.show()
#        img.save("output/img" + str(i) + ".jpg")
        


In [ ]:
!zip -r /content/output.zip /content/output
from google.colab import files
files.download("/content/output.zip")
 
!cp '/content/output.zip' '/content/drive/MyDrive/TFG/HTG/output.zip'